<a href="https://colab.research.google.com/github/koya-jp/AA-google-colab-kohya/blob/master/Diffusers_S2D2_ControlNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Diffusers ライブラリを用いて、画像を生成するスクリプト。**
参考 ：
[日本語](https://blog.shikoan.com/controlnet_lora/#ControlNet%EF%BC%88%E3%83%9D%E3%83%BC%E3%82%BA%EF%BC%89%E3%82%92%E4%BD%BF%E3%81%86)  ,
[英語](https://huggingface.co/blog/controlnet) ,
[Github - AA-google-colab-kohya](https://github.com/koya-jp/AA-google-colab-kohya/blob/master/Diffusers_S2D2.ipynb)

In [ ]:
#@title Driveに接続, ライブラリの追加 { display-mode: "form" }

# Driveに接続
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ライブラリの追加
!pip install --upgrade diffusers==0.17.1 transformers accelerate scipy ftfy safetensors txt2img >/dev/null 2>&1
!pip install k-diffusion >/dev/null 2>&1
!pip install --upgrade xformers git+https://github.com/huggingface/accelerate.git >/dev/null 2>&1
!pip install opencv-contrib-python >/dev/null 2>&1
!pip install controlnet_aux >/dev/null 2>&1

# 最新版（0.18.0）
# !pip install git+https://github.com/huggingface/diffusers

In [ ]:
#@title 自動切断されないようにするコード { display-mode: "form" }
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

In [ ]:
#@title ライブラリをインポート + S2D2 / Embeddingsの準備 { display-mode: "form" }
%cd /content
!git clone https://github.com/keisuke-okb/S2D2 &> /dev/null

%cd ./S2D2
!git pull
!touch __init__.py
!pip install -r requirements.txt >/dev/null 2>&1


# 各ライブラリをインポート
import s2d2

# diffusers ライブラリをインポートする
import diffusers
from diffusers import (StableDiffusionPipeline,StableDiffusionImg2ImgPipeline)

# PyTorchという深層学習フレームワークをインポートする
import torch


class StableDiffusionImageGeneratorMod(s2d2.StableDiffusionImageGenerator):
  def __init__(self, sd_safetensor_path: str, device: str="cuda", dtype: torch.dtype=torch.float16, controlnet=None, vae=None):
    self.device = torch.device(device)
    self.controlnet = controlnet # ここで属性を作る
    self.vae = vae
    self.pipe = StableDiffusionPipeline.from_pretrained(
      sd_safetensor_path,
      torch_dtype=dtype,
      custom_pipeline="lpw_stable_diffusion"
    ).to(device)
    self.pipe_i2i = StableDiffusionImg2ImgPipeline.from_pretrained(
      sd_safetensor_path,
      torch_dtype=dtype,
      custom_pipeline="lpw_stable_diffusion"
    ).to(device)
    self.pipe.safety_checker = None
    self.pipe_i2i.safety_checker = None
    return

  def load_embeddings(self, safetensors_path: str, fileName: str, token: str):
    if token != "":
      self.pipe.load_textual_inversion(safetensors_path, weight_name=fileName, token=token)
      self.pipe_i2i.load_textual_inversion(safetensors_path, weight_name=fileName, token=token)
    else:
      self.pipe.load_textual_inversion(safetensors_path, weight_name=fileName)
      self.pipe_i2i.load_textual_inversion(safetensors_path, weight_name=fileName)
    return

  %cd /content/


In [ ]:
#@title ControlNet の準備 { display-mode: "form" }

# ワイルドカードを使ってファイルパスを検索する
import glob
import os

# diffusers ライブラリをインポートする
from diffusers import (StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler, StableDiffusionImg2ImgPipeline) # + VAEModel (0.18.0)
from diffusers.utils import numpy_to_pil
from diffusers.pipelines.stable_diffusion import StableDiffusionPipelineOutput

# controlnet_auxというポーズ検出のライブラリをインポートする
from controlnet_aux import OpenposeDetector

# PyTorchという深層学習フレームワークをインポートする
import torch

# PILという画像処理ライブラリをインポートする
from PIL import Image


# JPEGファイルのパスのリストを取得
# @markdown ***ポージング学習用フォルダ***
dir = "/content/drive/MyDrive/StableDiffusion/ControlNet/spread_legs" #@param {type: "string"}
wildcard = "/*."
extension = "png" #@param {type: "string"}
images_path = os.path.join(dir, wildcard, extension)
jpeg_files = glob.glob(images_path)

# ポーズを検出する
controlnet_check_path = "lllyasviel/ControlNet" #@param {type: "string"}
pose_detector = OpenposeDetector.from_pretrained(controlnet_check_path) # 事前学習済みのモデルを読み込む
# ポーズのリストを作成する
poses = []
for file in jpeg_files:
    # ファイルパスから画像を読み込んでポーズを検出する
    p = pose_detector(Image.open(file))
    # ポーズをリストに追加する
    poses.append(p)

# コントロールネットワークを読み込む
controlnet_model_path = "lllyasviel/sd-controlnet-openpose"  #@param {type: "string"}
# 事前学習済みのモデルを読み込む。データ型はfloat16に指定する
controlnet = ControlNetModel.from_pretrained(controlnet_model_path, torch_dtype=torch.float16)



In [ ]:
# @title Model、LoRA を StableDiffusionImageGeneratorMod に設定 { display-mode: "form" }

#モデル
# @markdown **👚 Model ########################################################################**
model_path = "/content/drive/MyDrive/StableDiffusion/Model/chilled_remix_v2" #@param {type:"string"}


# VAE を読み込む
# vae_path = "stabilityai/sd-vae-ft-mse"  #@param {type:"string"}
# vae = VAEModel.from_pretrained(vae_path)

generator = StableDiffusionImageGeneratorMod(
  model_path,
  # vae=vae,
  controlnet=controlnet,
  device="cuda"
)

# @markdown **<br>🙎 LoRA ########################################################################**
LoRA_USE_1 = True #@param {type:"boolean"}
if LoRA_USE_1 == True:
  LoRA_1="/content/drive/MyDrive/StableDiffusion/Lora/flat2.safetensors" #@param {type:"string"}
  LoRA_alpha_1 = -1.0 #@param {type:"number"}
  generator.load_lora(LoRA_1, alpha=LoRA_alpha_1)

LoRA_USE_2= True #@param {type:"boolean"}
if LoRA_USE_2== True:
  LoRA_2="/content/drive/MyDrive/StableDiffusion/Lora/koreanDollLikeness_v20.safetensors" #@param {type:"string"}
  LoRA_alpha_2 = 0.6 #@param {type:"number"}
  generator.load_lora(LoRA_2, alpha=LoRA_alpha_2)

LoRA_USE_3= True #@param {type:"boolean"}
if LoRA_USE_3== True:
  LoRA_3="/content/drive/MyDrive/StableDiffusion/Lora/trueBuruma_v26red.safetensors" #@param {type:"string"}
  LoRA_alpha_3 = 0.1 #@param {type:"number"}
  generator.load_lora(LoRA_3, alpha=LoRA_alpha_3)

LoRA_USE_4= False #@param {type:"boolean"}
if LoRA_USE_4== True:
  LoRA_4="/content/drive/MyDrive/Lora/EkunePOVFellatioV2.safetensors" #@param {type:"string"}
  LoRA_alpha_4 = 0.5 #@param {type:"number"}
  generator.load_lora(LoRA_4, alpha=LoRA_alpha_4)


#@markdown **<br>👾 Embeddings ########################################################################**
embeddings_USE_1 = True #@param {type:"boolean"}
if embeddings_USE_1 == True:
  safetensors_path_1 = "/content/drive/MyDrive/StableDiffusion/embeddings/EasyNegative.safetensors" #@param {type:"string"}
  file_name_1 = "EasyNegative.safetensors" #@param {type:"string"}
  token_1 = "EasyNegative" #@param {type:"string"}
  generator.load_embeddings(safetensors_path_1, file_name_1, token_1)

embeddings_USE_2 = True #@param {type:"boolean"}
if embeddings_USE_2 == True:
  safetensors_path_2 = "/content/drive/MyDrive/StableDiffusion/embeddings/negative_hand-neg.pt" #@param {type:"string"}
  file_name_2 = "negative_hand-neg.pt" #@param {type:"string"}
  token_2 = "negative_hand-neg" #@param {type:"string"}
  generator.load_embeddings(safetensors_path_2, file_name_2, token_2)

embeddings_USE = True #@param {type:"boolean"}
if embeddings_USE == True:
  safetensors_path_3 = "/content/drive/MyDrive/StableDiffusion/embeddings/pureerosface_v1.pt" #@param {type:"string"}
  file_name_3 = "pureerosface_v1.pt" #@param {type:"string"}
  token_3 = "pureerosface_v1" #@param {type:"string"}
  generator.load_embeddings(safetensors_path_3, file_name_3, token_3)

embeddings_USE = True #@param {type:"boolean"}
if embeddings_USE == True:
  safetensors_path_4 = "/content/drive/MyDrive/StableDiffusion/embeddings/ng_deepnegative_v1_75t.pt" #@param {type:"string"}
  file_name_4 = "ng_deepnegative_v1_75t.pt" #@param {type:"string"}
  token_4 = "ng_deepnegative_v1_75t" #@param {type:"string"}
  generator.load_embeddings(safetensors_path_4, file_name_4, token_4)

# embeddings_USE = True #@param {type:"boolean"}
# if embeddings_USE == True:
#   safetensors_path_5 = "/content/drive/MyDrive/StableDiffusion/embeddings/negative_hand-neg.pt" #@param {type:"string"}
#   file_name_5 = "negative_hand-neg.pt" #@param {type:"string"}
#   token_5 = "negative_hand-neg" #@param {type:"string"}
#   generator.load_embeddings(safetensors_path_5, file_name_5, token_5)



In [ ]:
# @title 画像を生成  { display-mode: "form" }

# 必要なモジュールやライブラリをインポートする
import os
import random
import datetime

from PIL.PngImagePlugin import PngInfo

# ファイル名に使う日付と時刻のフォーマットを定義する
file_format = "%Y%m%d_%H%M%S"
i=0

# 現在の日本時間を取得
jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
#txt2img出力画像の保存先
#@markdown ***出力画像を保存するフォルダ***
save_path = "/content/drive/MyDrive/StableDiffusion/txt2img_output/test2/" #@param {type: "string"}
os.makedirs(save_path, exist_ok=True)


#@@markdown ポジティブプロンプト
prompt = "Beautiful girl,(See-through clothes very wet with water,sitting,very wet body:1.3),(red bloomers,red gym clothes, under size clothes),(camel toe, exposure pants:1.3 ),short brown hair,((super detail)), photo realistic, 8K UHD,(best quality:1. 2), high resolution, fine details, fine textures, facial expression, high details, delicate and beautiful eyes, perfectly proportioned face, light particles, clear_image, high resolution, high quality textures and shadows, realistic and beautiful face with big eyes, blush, glossy lips and perfect proportions, depth of field, lens flare, ray tracing, perspective,glossy skin, (smiling, shy),low angle,shiny hands, nfsw,see through nipples, see through pants" #@param {type:"string"}
#@@markdown ネガティブプロンプト
negative = "EasyNegative, painting, sketch, (worst picture quality:2), (low quality:2), (normal quality:2), bad feet, lowres, ((monochrome)),(ng_deepnegative_v1_75t),(negative_hand-neg:1.2),skin blemishes, acne, skin blemishes, bad anatomy, (bad fingers, bad hands, missing fingers:1.3), text, error, missing fingers, extra fingers, few digits, crop, worst picture quality, jpeg image, signature, watermark, Username, blurred, bad feet, (mutation, deformity:1. 3), extra limbs, fused fingers, long neck, crossed eyes, very low resolution, bad proportions" #@param {type:"string"}
#@markdown ***出力枚数***
batch_count = 20 #@param {type: "integer"}
#@markdown ***ステップ数***
steps = 20 #@param {type:"integer"}
#@@markdown 画像サイズ
img_width = "512" # @param [512, 768, 1024, 1536]
img_height = "768" # @param [512, 768, 1024, 1536]
#@@markdown CFG
CFG = 7 #@param {type: "number"}
#@markdown ***スケジューラ(サンプラー)***
scheduler="dpm++_2m_karras" #@param ["unipc","euler_a","euler","ddim","ddpm","deis","dpm2","dpm++_2s","dpm++_2m","dpm++_2m_karras","dpm++_sde","dpm++_sde_karras","heun","heun_karras","lms","lms_karras","pndm","dpm++_2m_karras"]
#@@markdown シード（-1の時はランダム）
seed = -1 #@param {type: "integer"}
if seed is None or seed == -1:
  inputSeed = random.randint(0, 2147483647)
else:
  valueSeed = seed

#@markdown ***Hires.fix の有効化***
hires_fix = True #@param {type: "boolean"}
enhance_steps = 2 if hires_fix else 1
#@markdown ***解像度倍率(乗算後、最も近い8の倍数のサイズとなる)***
upscaling_ratio = 2 #@param {type: "number"}
#@markdown ***アップスケールステップ数***
up_steps = 20 #@param {type: "integer"}
#@markdown ***小さいほど元画像を尊重） （0.5～0.7）が推奨***
denoising_strength = 0.60 #@param {type: "number"}


for i in range(batch_count):
  if seed is None or seed == -1:valueSeed = inputSeed + i
  image = generator.diffusion_enhance(
    prompt,
    negative,
    scheduler_name=scheduler,
    num_inference_steps=steps,
    num_inference_steps_enhance=up_steps,
    guidance_scale=CFG,
    width=img_width,
    height=img_height,
    seed=valueSeed,
    upscale_target="latent",
    interpolate_mode="bicubic",
    antialias=True,
    # diffusion_enhance=True,
    upscale_by=upscaling_ratio,
    enhance_steps=enhance_steps,
    denoising_strength=denoising_strength,
    output_type="pil",
    decode_factor=0.15,
    decode_factor_final=0.18215
  )

  # 現在の日本時間を取得
  jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
  #出力する画像の名前を生成する
  file_name = (jst_now.strftime(file_format)+ "_" + str(valueSeed))
  image_name = file_name + f".png"

  #画像を保存する
  save_location = os.path.join(save_path, image_name)

  #@markdown ***メタデータの書き込み***
  save_metadata = True #@param {type: "boolean"}
  if save_metadata:
    metadata = PngInfo()
    metadata.add_text("prompt",(prompt))
    metadata.add_text("negative",(negative))
    metadata.add_text("scheduler",(scheduler))
    metadata.add_text("steps",(str(steps)))
    metadata.add_text("CFG",(str(CFG)))
    metadata.add_text("width",(str(img_width)))
    metadata.add_text("height",(str(img_height)))
    metadata.add_text("seed",str((valueSeed)))
  if hires_fix:
    metadata.add_text("upscaling ratio",str((upscaling_ratio)))
    metadata.add_text("up steps",str((up_steps)))
    metadata.add_text("denoising strength",str((denoising_strength)))
    image.save(save_location, pnginfo=metadata)
  else:
    image.save(save_location)


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [13]:
# @title 画像のメタデータを出力

import sys
from PIL import Image
from PIL.PngImagePlugin import PngInfo

#@markdown **保存した画像のパス**
file_dir = "/content/drive/MyDrive/StableDiffusion/txt2img_output/test2/"  #@param {type: "string"}
file_name = "20230710_170302_1067497271.png" #@param {type: "string"}
file_path = os.path.join(save_path, file_name)
img = Image.open(file_path)

print("Prompt: ",img.text["prompt"])
print("Negative Prompt: ",img.text["negative"])
print("Scheduler: ", img.text["scheduler"])
print("Steps: ",img.text["steps"])
print("CFG: ",img.text["CFG"])
print("Width: ",img.text["width"])
print("Height: ",img.text["height"])
print("Seed: ",img.text["seed"])

try:
  print("Upscaling ratio; ",img.text["upscaling ratio"])
  print("Up steps: ",img.text["up steps"])
  print("Denoising strength: ",img.text["denoising strength"])
except:
  print("Hires.fix was OFF.")

# %%writefile output.txt

Prompt:  Beautiful girl on bed in love hotel,(See-through clothes very wet with water,very wet body:1.3),(red bloomers,red gym clothes, under size clothes),(camel toe, exposure pants:1.3 ),short brown hair,((super detail)), photo realistic, 8K UHD,(best quality:1. 2), high resolution, fine details, fine textures, facial expression, high details, delicate and beautiful eyes, perfectly proportioned face, light particles, clear_image, high resolution, high quality textures and shadows, realistic and beautiful face with big eyes, blush, glossy lips and perfect proportions, depth of field, lens flare, ray tracing, perspective,glossy skin, (smiling, shy),low angle,shiny hands, nfsw,see through nipples, see through pants
Negative Prompt:  EasyNegative, painting, sketch, (worst picture quality:2), (low quality:2), (normal quality:2), bad feet, lowres, ((monochrome)),(ng_deepnegative_v1_75t),(negative_hand-neg:1.2),skin blemishes, acne, skin blemishes, bad anatomy, (bad fingers, bad hands, miss